In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

--2023-10-02 08:44:04--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.41M  --.-KB/s    in 0.01s   

2023-10-02 08:44:04 (112 MB/s) - ‘data.csv’ saved [1475504/1475504]



In [2]:
ls

data.csv  sample_data/


In [292]:
import pandas as pd
import numpy as np

In [293]:
data = pd.read_csv('data.csv')

In [294]:
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [295]:
data = data[[
'Make',
'Model',
'Year',
'Engine HP',
'Engine Cylinders',
'Transmission Type',
'Vehicle Style',
'highway MPG',
'city mpg',
'MSRP']]

In [296]:
data = data.rename(columns = {'MSRP':'price'})

In [297]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

In [298]:
data.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
price                 0
dtype: int64

In [299]:
data = data.fillna(0)

In [300]:
data.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
price                0
dtype: int64

**Question 1: Most frequent value for `transmission_type`**

In [301]:
data['transmission_type'].value_counts()

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

**Question 2: Features with biggest correlation**

In [302]:
data.corr()

<ipython-input-302-c44ded798807>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [303]:
data['above_average'] = np.where(data.price > data.price.mean(), 1, 0)
data.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500,0


In [325]:
from sklearn.model_selection import train_test_split
df_train_full, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [326]:
len(df_train) / len(data), len(df_val) / len(data), len(df_test)/ len(data)

(0.5999664260533826, 0.2000167869733087, 0.2000167869733087)

In [327]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [328]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

In [329]:
del df_train['price']
del df_val['price']
del df_test['price']
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

**Question 3: Feature with the lowest mutual information score**

In [331]:
df_train_full.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
above_average          int64
dtype: object

In [332]:
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

categorical = ['make', 'model', 'transmission_type', 'vehicle_style']

In [333]:
df_train_full[categorical].nunique()

make                  48
model                902
transmission_type      5
vehicle_style         16
dtype: int64

In [334]:
from sklearn.metrics import mutual_info_score

In [335]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.above_average)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
#display(df_mi.tail())

,MI
model,0.460994
make,0.238724
vehicle_style,0.083390
transmission_type,0.020884


**Question 4: Training Logistic Regression**

In [336]:
from sklearn.feature_extraction import DictVectorizer

In [337]:
train_dict = df_train.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)

In [340]:
X_train.shape

(7148, 943)

In [341]:
from sklearn.linear_model import LogisticRegression

In [342]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [343]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [344]:
y_pred = model.predict(X_val)

In [350]:
orig_acc = (y_val == y_pred).mean()
np.round(orig_acc, 2)

0.95

**Question 5**

In [346]:
accuracies = dict()
features = categorical + numerical
for feature in range(len(features)):
      set_features = features[:feature] + features[feature+1:]
      train_dict = df_train[set_features].to_dict(orient='records')
      dv = DictVectorizer(sparse=False)
      dv.fit(train_dict)

      X_train = dv.transform(train_dict)

      model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
      model.fit(X_train, y_train)

      val_dict = df_val[set_features].to_dict(orient='records')
      X_val = dv.transform(val_dict)

      y_pred = model.predict(X_val)

      accuracy = (y_val == y_pred).mean()
      accuracies[features[feature]] = [accuracy, np.abs(orig_acc - accuracy)]

In [347]:
sorted(accuracies.items(), key=lambda x:x[1][1])

[('engine_hp', [0.9349559378934117, 0.00041963911036513313]),
 ('vehicle_style', [0.9320184641208561, 0.0025178346621904657]),
 ('transmission_type', [0.9450272765421738, 0.010490977759127218]),
 ('city_mpg', [0.9458665547629039, 0.011330255979857373]),
 ('highway_mpg', [0.9467058329836341, 0.012169534200587528]),
 ('engine_cylinders', [0.9471254720939991, 0.01258917331095255]),
 ('year', [0.9483843894250944, 0.013848090642047839]),
 ('make', [0.9488040285354595, 0.014267729752412972]),
 ('model', [0.9164918170373478, 0.018044481745698726])]

**Question 6**

In [351]:
from sklearn.linear_model import Ridge

In [ ]:
[0, 0.01, 0.1, 1, 10]

In [404]:
data.price = np.log1p(data.price)

In [407]:
del data['above_average']

In [408]:
data.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,0.807422
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,0.806022
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,0.804768
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,0.802359
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,0.804176


In [432]:
df_train_full, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [433]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [434]:
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

In [435]:
del df_train['price']
del df_val['price']
del df_test['price']

In [436]:
X_train = data.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(X_train)

X_train = dv.transform(X_train)

In [437]:
X_train.shape

(11914, 990)

In [438]:
val_dict = df_val.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(val_dict)

X_val = dv.transform(val_dict)
X_val.shape

(2383, 788)

In [422]:
clf = Ridge(alpha=0.01, solver = 'sag', random_state=42)

In [427]:
clf.fit(X_tr, y_train)

ValueError: ignored

In [ ]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [373]:
y_pred = clf.predict(X_val)

ValueError: ignored